In [2]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import requests
from bs4 import BeautifulSoup
import time
import lxml
import re
import string

# Parsing XML Files from ProPublica

Last updated: 6/1/23 by Christopher Lum

### Methods

`scrape_website(url, attempts=0)`
- Takes a URL and returns the response object if successful. Retries three times on failure

`info_grabber(fp, url=True)`
- Takes the URL/filepath of an XML file. The `url` parameter indicates whether the given filepath is a URL or local
- Returns a dictionary of the row information for section VIII (along with a couple other columns)
- Older (prior to 2013) XML files have different tags and thus can't be read. Results in a row of all zeroes and `'Not Found'` in the `ba` column
- Of successfully scraped columns, has ~1.5% error rate for rows (0.13% of cells are inaccurate), only in the `Fundraising Events` column due to inconsistent labeling in XMLs
    - This error rate is lower than the human error rate
- In 2013, they had a different way of tagging zip codes, so all 2013 zips will be 0

`contact_grabber(fp, url=True)`
- Returns a dataframe of each contact in the 990 form, according to section VII
- Incomplete and edge cases aren't tested fully, as we didn't need the function.

`get_xmls(ein)`
- Takes the EIN of a company and returns a list of the URLs for each XML file that is linked on the ProPublica page for that given company

`grabber(eins, verbose=False, clean=False)`
- Takes a list of EINs and uses `get_xmls()` to find each XML file, then uses `info_grabber()` to scrape each XML for the relevant information.
- Returns a pandas DataFrame, doesn't automatically save as csv.
- Takes around 22 minutes on my laptop for 71 businesses
- The `clean` parameter automatically cleans the dataframe by removing empty rows

`find_errors(info)`
- Data columns are supposed to add up to the total, if it doesn't this method will find it
- Takes a dataframe of 990 rows from `grabber` and returns a tuple of incides that didn't properly grab all of the data
    - Preferably indiced on `EIN_YEAR`

#### `scrape_website`

In [3]:
def scrape_website(url, attempts=0):
    # Send GET request to the website
    response = requests.get(url)
    time.sleep(1)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        if attempts > 0:
            print('Success')
        return response
    elif attempts < 3:
        print("Error:", response.status_code, "Trying again.")
        time.sleep(1)
        return scrape_website(url, attempts + 1)
    else:
        # Handle the request error
        print("Error: ", response.status_code, 'Failed on:', url)
        return None


#### `info_grabber`

In [4]:
def info_grabber(fp, url=True, errors=False):
    if url:
        xml = scrape_website(fp)
        xml.encoding = 'UTF-8'
        xml = xml.text.strip()
    else:
        try:
            with open(fp, 'r') as file:
                xml = file.read()
        except FileNotFoundError:
            print(f"File not found: {fp}")
            return dict()
    if xml is None:
        return dict()
    
    soup = BeautifulSoup(xml, 'xml')

    row = dict()
    row['ba'] = soup.find('BusinessName')
    row['EIN'] = soup.find('EIN')
    row['Tax Year'] = soup.find('TaxPeriodBeginDt')
    row['Tax End Date'] = soup.find('TaxPeriodEndDt')
    row['Location (Zipcode)'] = soup.find('USAddress')
    row['Federate Campaigns'] = soup.find('FederatedCampaignsAmt')
    row['Membership Dues'] = soup.find('MembershipDuesAmt')
    row['Fundraising Events'] = soup.find('ContriRptFundraisingEventAmt')
    
    row['Related Organizations'] = soup.find('RelatedOrganizationsAmt')
    row['Government Grants'] = soup.find('GovernmentGrantsAmt')
    row['All Other Contributions'] = soup.find('AllOtherContributionsAmt')
    row['Noncash Contributions'] = soup.find('NoncashContributionsAmt')
    row['Total'] = soup.find('TotalContributionsAmt')
    # Can make default=0

    if row['Location (Zipcode)'] is not None:
        row['Location (Zipcode)'] = row['Location (Zipcode)'].find('ZIPCd')
    else:
        row['Location (Zipcode)'] = None

    row = dict(map(lambda item: (item[0], item[1].text if item[1] is not None else None), row.items()))

    if row['ba'] is not None:
        row['ba'] = row['ba'].strip()
    else:
        row['ba'] = 'Not found'
    if row['Tax Year'] is not None:
        row['Tax Year'] = pd.Timestamp(row['Tax Year']).year
    else:
        row['Tax Year'] = np.nan
    if row['Tax End Date'] is not None:
        row['Tax End Date'] = pd.to_datetime(row['Tax End Date']).strftime("%m-%d-%Y")
    else:
        row['Tax End Date'] = np.nan
    
    if errors:
        sumKeys = ['Federate Campaigns', 'Membership Dues', 'Related Organizations', 'Government Grants', 'All Other Contributions']
        sumAmt = sum(float(row[key]) for key in sumKeys if key in row and row[key] is not None)
        fundAmt_element = soup.find('FundraisingAmt')
        fundAmt = int(fundAmt_element.text) if fundAmt_element is not None else 0
        sumAmt += fundAmt

        if sumAmt is None or row['Total'] is None:
            pass
        elif int(sumAmt) == int(row['Total']):
            row['Fundraising Events'] = fundAmt
        else:
            pass

    row = dict(map(lambda item: (item[0], item[1] if item[1] is not None else 0), row.items()))

    return row


#### `contact_grabber`

Old, might not work

In [5]:
def contact_grabber(fp, url=True):
    '''
    Return a dataframe of the important names and their positions in an organization for each year

    :param fp: filepath of the xml file
    :param url: Whether the fp is a url or not. Default = True
    '''
    if url:
        xml = scrape_website(fp)
    else:
        xml = requests.get(fp)
    if xml is None:
        return dict()
    xml.encoding = 'UTF-8'
    xml = xml.text.strip()
    soup = BeautifulSoup(xml, 'xml')

    people = soup.find_all('Form990PartVIISectionAGrp')
    names = [x.find('PersonNm').text for x in people if x.find('PersonNm') is not None]
    titles = [x.find('TitleTxt').text for x in people if x.find('TitleTxt') is not None]
    try:
        organization = [soup.find('BusinessName').text.strip()] * len(names)
    except AttributeError:
        organization = ['Not found'] * len(names)
    try:
        year = [pd.Timestamp(soup.find('TaxPeriodBeginDt').text).year] * len(names)
    except AttributeError:
        year = [0] * len(names)
        
    return pd.DataFrame({'organization': organization, 'year': year, 'names': names, 'titles': titles})

#### `get_xmls`

In [23]:
def get_xmls(ein):
    '''
    Return a list of xml urls from ProRepublica based on an organization's EIN

    :param ein: Employer Identification number as String or Int
    '''
    url = f'https://projects.propublica.org/nonprofits/organizations/{ein}'
    html = scrape_website(url)
    if html is None:
        return []
    soup = BeautifulSoup(html.text)
    links = soup.find_all('a', class_='action xml')
    base_url = 'https://projects.propublica.org'
    urls = [base_url + x.get('href') for x in links if x.text == '990']
    urls = [x for x in soup.find_all(class_='action xml') if re.search(r'990\b', x.text)]
    urls = [base_url + x.get('href') if x.name == 'a' else base_url + x.select_one('select.action.xml option[data-href]').get('data-href') for x in urls]
    return urls

#### `grabber`

In [7]:
def grabber(eins, verbose=False, clean=False, errors=False):
    if type(eins) == int:
        eins = [eins]
    data = []
    # contacts = pd.DataFrame()
    overall_index = 0
    for index, ein in enumerate(eins):
        print(str(index) + (' / ') + str(len(eins)))
        xmls = get_xmls(ein)
        for xml in xmls:
            if verbose:
                print(f'{overall_index} {xml}')
            if errors:
                data.append(info_grabber(xml,errors=True))
            else:
                data.append(info_grabber(xml))
            #contacts = pd.concat([contacts, contact_grabber(xml)])
            overall_index += 1
    print(str(len(eins)) + (' / ') + str(len(eins)))
    info = pd.DataFrame(data)
    if not clean:
        return info

    no_error = info.dropna().reset_index(drop=True)
    no_error.loc[:, 'EIN':'Total'] = no_error.loc[:, 'EIN':'Total'].astype(int)
    no_error.loc[:, 'EIN_YEAR'] = no_error['EIN'].astype(str) + '_' + no_error['Tax Year'].astype(str)
    no_error = no_error.set_index('EIN_YEAR')

    return no_error

#### `find_errors`

In [8]:
def find_errors(info):
    '''
    Takes a dataframe of 990 rows and returns a tuple of the indices where it failed to grab the data properly

    :param info: The dataframe of 990 rows
    '''
    accurate = info[['Federate Campaigns', 'Membership Dues', 'Fundraising Events', 'Related Organizations', 'Government Grants', 'All Other Contributions']].sum(axis=1) == info['Total']
    return tuple(accurate[~accurate].index)

### Using the `grabber()` method

`grabber()` takes a list of EINs. This was curated from the old "990 Manual Pull" spreadsheet by pasting and splitting the entire `EIN` column and putting it into a set.

In [9]:
all_eins = '''113723093
113723093
204374795
237161267
237279074
237334012
261712580
261712580
263405689
270447059
270447059
270865318
270865318
272917644
273390797
330006089
330008269
330008269
330122462
330210280
330215585
330215585
330217339
330317950
330433314
330492304
330492304
330496092
330497515
330553621
330553621
330553621
330602842
330602842
330618893
330623634
330902617
330902617
331029843
331029843
331146733
331146733
465055513
465055513
550806460
562613191
571162424
571162424
571162424
571214920
571214920
571214920
821946283
822363154
931008369
941676390
942358632
942358632
951644024
951648184
951869806
951874073
951944230
951945256
952039198
952039198
952111196
952157559
952157559
952213995
952422704
952422704
952457354
952457354
952648050
952653869
952693142
952693142
952794253
952794253
952794253
952833205
952833205
952833205
952850121
952877102
952880426
952880426
952949636
953031682
953138268
953138268
953140767
953140767
953244085
953244085
953248148
953248148
953302967
953302967
953315571
953368020
953497926
953649525
953649525
953750738
953782164
953782164
953798088
953837714
953837714
953950196
953950196
953950196
956379598
204374795
311750839
330553704
330498557
880279721
330352148
330352148
951729734
237332048'''

In [10]:
wrong_eins = '''952850121 562613191 952457354 330210280 952877102 330433314 204374795 330602842 465055513 263405689 263405689 330602842 330602842 330602842 330433314 951644024 330553621 952653869 330433314 941676390 953302967 941676390 941676390 941676390 465055513 330602842 952850121 330006089 952111196 330433314 562613191 952880426 953244085
'''

In [11]:
eins = list(set(all_eins.split()))
# eins = list(set(wrong_eins.split()))

<b>Warning</b>: `grabber()` takes a long time. The following cell will take about 20 minutes.

# 990 - Manual Pull Testing

In [12]:
info990 = grabber(eins, verbose=True)

0 / 83
0 https://projects.propublica.org/nonprofits/download-xml?object_id=202241319349303914
1 https://projects.propublica.org/nonprofits/download-xml?object_id=202111319349302666
2 https://projects.propublica.org/nonprofits/download-xml?object_id=202010579349301766
3 https://projects.propublica.org/nonprofits/download-xml?object_id=201910379349301136
4 https://projects.propublica.org/nonprofits/download-xml?object_id=201801289349302885
5 https://projects.propublica.org/nonprofits/download-xml?object_id=201720479349300032
6 https://projects.propublica.org/nonprofits/download-xml?object_id=201610909349301051
7 https://projects.propublica.org/nonprofits/download-xml?object_id=201501209349301140
1 / 83
8 https://projects.propublica.org/nonprofits/download-xml?object_id=202201369349318250
9 https://projects.propublica.org/nonprofits/download-xml?object_id=202140089349302069
10 https://projects.propublica.org/nonprofits/download-xml?object_id=201923109349302212
11 https://projects.propubli

93 https://projects.propublica.org/nonprofits/download-xml?object_id=201843189349312029
94 https://projects.propublica.org/nonprofits/download-xml?object_id=201713129349301956
95 https://projects.propublica.org/nonprofits/download-xml?object_id=201623199349306427
96 https://projects.propublica.org/nonprofits/download-xml?object_id=201532249349301358
97 https://projects.propublica.org/nonprofits/download-xml?object_id=201422279349302317
98 https://projects.propublica.org/nonprofits/download-xml?object_id=201332199349301133
99 https://projects.propublica.org/nonprofits/download-xml?object_id=201243189349300544
100 https://projects.propublica.org/nonprofits/download-xml?object_id=201123199349302942
11 / 83
101 https://projects.propublica.org/nonprofits/download-xml?object_id=202231749349301418
102 https://projects.propublica.org/nonprofits/download-xml?object_id=202131959349300828
103 https://projects.propublica.org/nonprofits/download-xml?object_id=202041959349302419
104 https://projects

185 https://projects.propublica.org/nonprofits/download-xml?object_id=201940249349301019
186 https://projects.propublica.org/nonprofits/download-xml?object_id=201800269349301550
187 https://projects.propublica.org/nonprofits/download-xml?object_id=201710269349300021
188 https://projects.propublica.org/nonprofits/download-xml?object_id=201640329349300984
189 https://projects.propublica.org/nonprofits/download-xml?object_id=201520289349301172
190 https://projects.propublica.org/nonprofits/download-xml?object_id=201400239349301215
191 https://projects.propublica.org/nonprofits/download-xml?object_id=201320329349300962
192 https://projects.propublica.org/nonprofits/download-xml?object_id=201230909349300238
193 https://projects.propublica.org/nonprofits/download-xml?object_id=201120329349300712
19 / 83
194 https://projects.propublica.org/nonprofits/download-xml?object_id=202330819349301668
195 https://projects.propublica.org/nonprofits/download-xml?object_id=202201369349311415
196 https://p

29 / 83
30 / 83
277 https://projects.propublica.org/nonprofits/download-xml?object_id=202300769349300130
278 https://projects.propublica.org/nonprofits/download-xml?object_id=202221329349304867
279 https://projects.propublica.org/nonprofits/download-xml?object_id=202100649349300545
280 https://projects.propublica.org/nonprofits/download-xml?object_id=202000709349300040
281 https://projects.propublica.org/nonprofits/download-xml?object_id=201920809349300542
282 https://projects.propublica.org/nonprofits/download-xml?object_id=201810689349300446
283 https://projects.propublica.org/nonprofits/download-xml?object_id=201701329349302285
284 https://projects.propublica.org/nonprofits/download-xml?object_id=201631339349302733
285 https://projects.propublica.org/nonprofits/download-xml?object_id=201500759349301170
286 https://projects.propublica.org/nonprofits/download-xml?object_id=201431139349300913
287 https://projects.propublica.org/nonprofits/download-xml?object_id=201320739349301012
288 h

369 https://projects.propublica.org/nonprofits/download-xml?object_id=201811359349307736
370 https://projects.propublica.org/nonprofits/download-xml?object_id=201731289349302128
371 https://projects.propublica.org/nonprofits/download-xml?object_id=201641189349300539
372 https://projects.propublica.org/nonprofits/download-xml?object_id=201500989349300900
40 / 83
373 https://projects.propublica.org/nonprofits/download-xml?object_id=202202279349302275
374 https://projects.propublica.org/nonprofits/download-xml?object_id=202112249349302051
375 https://projects.propublica.org/nonprofits/download-xml?object_id=202022309349302537
376 https://projects.propublica.org/nonprofits/download-xml?object_id=201922219349300012
377 https://projects.propublica.org/nonprofits/download-xml?object_id=201841769349301204
378 https://projects.propublica.org/nonprofits/download-xml?object_id=201721539349300132
379 https://projects.propublica.org/nonprofits/download-xml?object_id=201631419349300728
380 https://p

461 https://projects.propublica.org/nonprofits/download-xml?object_id=201640359349301069
462 https://projects.propublica.org/nonprofits/download-xml?object_id=201540439349301519
463 https://projects.propublica.org/nonprofits/download-xml?object_id=201420139349302757
49 / 83
464 https://projects.propublica.org/nonprofits/download-xml?object_id=202232909349300938
465 https://projects.propublica.org/nonprofits/download-xml?object_id=202142499349300204
466 https://projects.propublica.org/nonprofits/download-xml?object_id=202012689349300236
467 https://projects.propublica.org/nonprofits/download-xml?object_id=201931639349300028
468 https://projects.propublica.org/nonprofits/download-xml?object_id=201801919349300140
469 https://projects.propublica.org/nonprofits/download-xml?object_id=201710939349301226
470 https://projects.propublica.org/nonprofits/download-xml?object_id=201621329349303557
50 / 83
471 https://projects.propublica.org/nonprofits/download-xml?object_id=202233069349302923
472 h

553 https://projects.propublica.org/nonprofits/download-xml?object_id=201841239349300224
554 https://projects.propublica.org/nonprofits/download-xml?object_id=201720969349301107
555 https://projects.propublica.org/nonprofits/download-xml?object_id=201641279349301054
556 https://projects.propublica.org/nonprofits/download-xml?object_id=201223149349300607
557 https://projects.propublica.org/nonprofits/download-xml?object_id=201103079349300000
59 / 83
558 https://projects.propublica.org/nonprofits/download-xml?object_id=202311359349316171
559 https://projects.propublica.org/nonprofits/download-xml?object_id=202211229349302251
560 https://projects.propublica.org/nonprofits/download-xml?object_id=202120859349301007
561 https://projects.propublica.org/nonprofits/download-xml?object_id=202031909349302653
562 https://projects.propublica.org/nonprofits/download-xml?object_id=201921349349310562
563 https://projects.propublica.org/nonprofits/download-xml?object_id=201811359349309706
564 https://p

645 https://projects.propublica.org/nonprofits/download-xml?object_id=201213199349304261
646 https://projects.propublica.org/nonprofits/download-xml?object_id=201201249349300815
647 https://projects.propublica.org/nonprofits/download-xml?object_id=201100839349300115
70 / 83
648 https://projects.propublica.org/nonprofits/download-xml?object_id=202340839349301104
649 https://projects.propublica.org/nonprofits/download-xml?object_id=202213199349329616
650 https://projects.propublica.org/nonprofits/download-xml?object_id=202033219349300928
651 https://projects.propublica.org/nonprofits/download-xml?object_id=201923199349318877
652 https://projects.propublica.org/nonprofits/download-xml?object_id=201823199349320347
653 https://projects.propublica.org/nonprofits/download-xml?object_id=201723199349317287
654 https://projects.propublica.org/nonprofits/download-xml?object_id=201613209349309136
655 https://projects.propublica.org/nonprofits/download-xml?object_id=201502339349300610
656 https://p

737 https://projects.propublica.org/nonprofits/download-xml?object_id=201440459349301904
738 https://projects.propublica.org/nonprofits/download-xml?object_id=201310399349301016
739 https://projects.propublica.org/nonprofits/download-xml?object_id=201211369349301826
740 https://projects.propublica.org/nonprofits/download-xml?object_id=201121369349304167
80 / 83
741 https://projects.propublica.org/nonprofits/download-xml?object_id=202301359349316195
742 https://projects.propublica.org/nonprofits/download-xml?object_id=202232109349301663
743 https://projects.propublica.org/nonprofits/download-xml?object_id=202140429349300029
744 https://projects.propublica.org/nonprofits/download-xml?object_id=202001339349300630
745 https://projects.propublica.org/nonprofits/download-xml?object_id=201941069349300329
746 https://projects.propublica.org/nonprofits/download-xml?object_id=201800449349301180
747 https://projects.propublica.org/nonprofits/download-xml?object_id=201700419349301365
748 https://p

In [13]:
info990

,ba,EIN,Tax Year,Tax End Date,Location (Zipcode),Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,All Other Contributions,Noncash Contributions,Total
0,SAN DIEGO YOUTH SERVICES,952648050,2020.0,06-30-2021,92110,0,0,0,0,19387646,322601,0,19710247
1,SAN DIEGO YOUTH SERVICES,952648050,2019.0,06-30-2020,92110,0,0,0,0,19632420,203870,0,19836290
2,SAN DIEGO YOUTH SERVICES,952648050,2018.0,06-30-2019,92110,0,0,0,0,19070850,144882,0,19215732
3,SAN DIEGO YOUTH SERVICES,952648050,2017.0,06-30-2018,92110,0,0,0,0,17914417,225235,0,18139652
4,SAN DIEGO YOUTH SERVICES,952648050,2016.0,06-30-2017,92110,0,0,0,0,0,339948,0,339948
...,...,...,...,...,...,...,...,...,...,...,...,...,...
756,THINK DIGNITY,331146733,2019.0,12-31-2019,92104,0,0,48628,0,129001,125440,43807,303069
757,THINK DIGNITY,331146733,2018.0,12-31-2018,92104,0,0,33682,0,114747,324226,25426,472655
758,THINK DIGNITY,331146733,2017.0,12-31-2017,92104,0,4925,11158,0,106172,255771,0,381968
759,Think Dignity,331146733,2016.0,12-31-2016,92104,0,3445,20231,0,121084,114551,11272,259311


### Checking the missing cells

The following two cells should both be 1.0.

The first checks the conditional probability that if the name is not found in a row, the tax year wasn't found.

The second checks the inverse conditional probability. Sometimes, if the request attempt tried and failed three times, the row will be entirely empty, where the name is `''` and not `'Not Found'`

In [12]:
# Every time the name isn't found, nothing is found
info990.loc[info990['ba'] == 'Not found']['Tax Year'].isna().mean()

0.0

In [13]:
# And every time the tax year isn't found the name isn't found
(info990[['ba', 'Tax Year']].loc[info990['Tax Year'].isna()]['ba'] == 'Not found').mean()

nan

This cell indicates how many XML files failed to be accessed

In [14]:
info990['ba'].isna().sum()

0

If the following cell returns 0.0, that indicates that every time the tax year is missing, every single financial cell is zero. All of these rows can be dropped and no information is lost.

In [15]:
info990.loc[info990['Tax Year'].isna(), 'Federate Campaigns':].sum(axis=1).sum()

0

### Cleaning the raw information

Assuming the prior three checks are all adequate, we can drop rows with any nan values.

In [14]:
no_error = info990.dropna().reset_index(drop=True)
int_columns = no_error.columns.difference(['ba','Tax End Date'])  # Skip the 'Tax End Date' column
no_error[int_columns] = no_error[int_columns].astype(int)
no_error['EIN_YEAR'] = no_error['EIN'].astype(str) + '_' + no_error['Tax Year'].astype(str)
no_error = no_error.set_index('EIN_YEAR')

### Export here!

<b>REMEMBER TO EXPORT</b>

`no_error` is the cleaned dataframe

`info990` is the raw dataframe with missing rows

In [80]:
 no_error.to_csv('no_errors.csv')

In [18]:
 info990.to_csv('full_table.csv')

The following line appends the rows to the bottom of the existing dataframe titled `'no_errors.csv'` instead.

It creates a new CSV and it is recommended to rename it back to `no_errors.csv` in the case it is exported successfully

In [81]:
extended = pd.concat([pd.read_csv('no_errors.csv'), no_error]).reset_index(drop=True).drop_duplicates()
extended.to_csv('no_errors_appended.csv')

## Testing Zone

In [ ]:
url = 'https://projects.propublica.org/nonprofits/organizations/263405689'
html = scrape_website(url)
soup = BeautifulSoup(html.text)
links = soup.find_all('a', class_='action xml')
base_url = 'https://projects.propublica.org'
urls = [base_url + x.get('href') for x in links if x.text == '990']

In [21]:
test = [x for x in soup.find_all(class_='action xml') if re.search(r'990\b', x.text)]
test2 = [base_url + x.get('href') if x.name == 'a' else base_url + x.select_one('select.action.xml option[data-href]').get('data-href') for x in test]

In [ ]:
info990

In [15]:
extended = pd.concat([pd.read_csv('no_errors.csv'), no_error]).reset_index(drop=True).drop_duplicates()

In [84]:
info = pd.read_csv('no_errors.csv')

In [85]:
info = info.set_index('EIN_YEAR')
# accurate = info[['Federate Campaigns', 'Membership Dues', 'Fundraising Events', 'Related Organizations', 'Government Grants', 'All Other Contributions']].sum(axis=1) == info['Total']

In [28]:
info

,ba,EIN,Tax Year,Tax End Date,Location (Zipcode),Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,All Other Contributions,Noncash Contributions,Total
EIN_YEAR,,,,,,,,,,,,,
942358632_2021,WALDEN ENVIRONMENT\nDBA WALDEN FAMILY SERVICES,942358632,2021,1970.0,92123,0,0,366680,0,16561736,1186451,10000,18114867
942358632_2020,WALDEN ENVIRONMENT\nDBA WALDEN FAMILY SERVICES,942358632,2020,1970.0,92123,0,0,0,0,17100546,833046,3100,17933592
942358632_2019,WALDEN ENVIRONMENT INC\nDBA WALDEN FAMILY SERV...,942358632,2019,1970.0,92123,0,0,1071693,0,16055428,658669,35545,17785790
942358632_2018,WALDEN ENVIRONMENT INC\nDBA WALDEN FAMILY SERV...,942358632,2018,1970.0,92123,0,0,1007674,0,13353015,441793,681176,14802482
942358632_2017,WALDEN ENVIRONMENT INC\nDBA WALDEN FAMILY SERV...,942358632,2017,1970.0,92123,0,0,0,0,0,252500,0,252500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
330618893_2018,ESCONDIDO GARDENS HOUSING FOUNDATION,330618893,2018,1970.0,92127,0,0,0,0,59823,0,0,59823
330618893_2017,ESCONDIDO GARDENS HOUSING FOUNDATION,330618893,2017,1970.0,92127,0,0,0,0,59524,0,0,59524
330618893_2016,ESCONDIDO GARDENS HOUSING FOUNDATION,330618893,2016,1970.0,92127,0,0,0,0,58030,0,0,58030


In [29]:
# tuple(accurate[~accurate].index)

In [26]:
info_EINerrs = find_errors(no_error)
info_EINerrs

('550806460_2022',
 '953302967_2013',
 '952039198_2020',
 '952039198_2013',
 '953497926_2016',
 '272917644_2017',
 '951874073_2015',
 '263405689_2015',
 '953368020_2015',
 '571214920_2016',
 '331146733_2017')

In [87]:
info_errs = info[info.index.isin(info_EINerrs)]

In [27]:
info_errs = [int(err[:-5]) for (err) in info_EINerrs]
info_errs

[550806460,
 953302967,
 952039198,
 952039198,
 953497926,
 272917644,
 951874073,
 263405689,
 953368020,
 571214920,
 331146733]

In [28]:
filtered_info_errs = grabber(info_errs, verbose=True, errors=True)

0 / 11
0 https://projects.propublica.org/nonprofits/download-xml?object_id=202320899349301857
1 https://projects.propublica.org/nonprofits/download-xml?object_id=202221389349301022
2 https://projects.propublica.org/nonprofits/download-xml?object_id=202110749349300046
3 https://projects.propublica.org/nonprofits/download-xml?object_id=202022669349300022
4 https://projects.propublica.org/nonprofits/download-xml?object_id=201932909349301313
5 https://projects.propublica.org/nonprofits/download-xml?object_id=201833199349312273
6 https://projects.propublica.org/nonprofits/download-xml?object_id=201733569349300623
7 https://projects.propublica.org/nonprofits/download-xml?object_id=201633159349303143
1 / 11
8 https://projects.propublica.org/nonprofits/download-xml?object_id=202310489349300246
9 https://projects.propublica.org/nonprofits/download-xml?object_id=202211369349314141
10 https://projects.propublica.org/nonprofits/download-xml?object_id=202140129349300914
11 https://projects.propubli

93 https://projects.propublica.org/nonprofits/download-xml?object_id=201723179349307002
94 https://projects.propublica.org/nonprofits/download-xml?object_id=201741379349300649
95 https://projects.propublica.org/nonprofits/download-xml?object_id=201643209349307094
9 / 11
96 https://projects.propublica.org/nonprofits/download-xml?object_id=202331379349301638
97 https://projects.propublica.org/nonprofits/download-xml?object_id=202220949349300532
98 https://projects.propublica.org/nonprofits/download-xml?object_id=202111369349300141
99 https://projects.propublica.org/nonprofits/download-xml?object_id=201900809349300545
100 https://projects.propublica.org/nonprofits/download-xml?object_id=201821359349312922
10 / 11
101 https://projects.propublica.org/nonprofits/download-xml?object_id=202301199349300100
102 https://projects.propublica.org/nonprofits/download-xml?object_id=202212309349301596
103 https://projects.propublica.org/nonprofits/download-xml?object_id=202142319349301129
104 https://p

In [29]:
filtered_info_errs

,ba,EIN,Tax Year,Tax End Date,Location (Zipcode),Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,All Other Contributions,Noncash Contributions,Total
0,Bilateral Safety Corridor Coalition,550806460,2022.0,12-31-2022,91950,0,0,8407,0,291440,270209,0,570056
1,Bilateral Safety Corridor Coalition,550806460,2021.0,12-31-2021,91950,0,0,0,0,239763,227448,0,467211
2,Bilateral Safety Corridor Coalition,550806460,2020.0,12-31-2020,91950,0,0,0,0,161068,218547,6000,379615
3,Bilateral Safety Corridor Coalition,550806460,2019.0,12-31-2019,91950,0,0,0,0,0,269047,0,269047
4,BILATERAL SAFETY CORRIDOR COALITION,550806460,2018.0,12-31-2018,91950,0,0,0,0,0,297479,0,297479
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,THINK DIGNITY,331146733,2019.0,12-31-2019,92104,0,0,48628,0,129001,125440,43807,303069
105,THINK DIGNITY,331146733,2018.0,12-31-2018,92104,0,0,33682,0,114747,324226,25426,472655
106,THINK DIGNITY,331146733,2017.0,12-31-2017,92104,0,4925,15100,0,106172,255771,0,381968
107,Think Dignity,331146733,2016.0,12-31-2016,92104,0,3445,20231,0,121084,114551,11272,259311


In [50]:
info_no_errors = filtered_info_errs.dropna().reset_index(drop=True)
int_columns = info_no_errors.columns.difference(['ba','Tax End Date'])  # Skip the 'Tax End Date' column
info_no_errors[int_columns] = info_no_errors[int_columns].astype(int)
info_no_errors['EIN_YEAR'] = info_no_errors['EIN'].astype(str) + '_' + info_no_errors['Tax Year'].astype(str)
info_no_errors = info_no_errors.set_index('EIN_YEAR')
info_no_errors

,ba,EIN,Tax Year,Tax End Date,Location (Zipcode),Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,All Other Contributions,Noncash Contributions,Total
EIN_YEAR,,,,,,,,,,,,,
550806460_2022,Bilateral Safety Corridor Coalition,550806460,2022,12-31-2022,91950,0,0,8407,0,291440,270209,0,570056
550806460_2021,Bilateral Safety Corridor Coalition,550806460,2021,12-31-2021,91950,0,0,0,0,239763,227448,0,467211
550806460_2020,Bilateral Safety Corridor Coalition,550806460,2020,12-31-2020,91950,0,0,0,0,161068,218547,6000,379615
550806460_2019,Bilateral Safety Corridor Coalition,550806460,2019,12-31-2019,91950,0,0,0,0,0,269047,0,269047
550806460_2018,BILATERAL SAFETY CORRIDOR COALITION,550806460,2018,12-31-2018,91950,0,0,0,0,0,297479,0,297479
...,...,...,...,...,...,...,...,...,...,...,...,...,...
331146733_2019,THINK DIGNITY,331146733,2019,12-31-2019,92104,0,0,48628,0,129001,125440,43807,303069
331146733_2018,THINK DIGNITY,331146733,2018,12-31-2018,92104,0,0,33682,0,114747,324226,25426,472655
331146733_2017,THINK DIGNITY,331146733,2017,12-31-2017,92104,0,4925,15100,0,106172,255771,0,381968


In [51]:
find_errors(info_no_errors)

('951874073_2015',)

In [33]:
info_no_errors.to_csv('info_no_errors.csv')
EINinfoerrs = pd.read_csv('info_no_errors.csv')
EINinfoerrs = EINinfoerrs.set_index('EIN_YEAR')

# Testing EINs from Manual Pull

In [ ]:
csvfile_eins = pd.read_csv('990 Manual Pull - Cleaned EIN List.csv')
eins = list(csvfile_eins.get('EIN').dropna())
eins = [int(ein) for ein in eins]
#eins

In [44]:
parsedEINs = grabber(eins)#, verbose=True)

In [53]:
no_errorsEIN = parsedEINs.dropna().reset_index(drop=True)
int_columns = no_errorsEIN.columns.difference(['ba','Tax End Date'])  # Skip the 'Tax End Date' column
no_errorsEIN[int_columns] = no_errorsEIN[int_columns].astype(int)
no_errorsEIN['EIN_YEAR'] = no_errorsEIN['EIN'].astype(str) + '_' + no_errorsEIN['Tax Year'].astype(str)
no_errorsEIN.set_index('EIN_YEAR')

,ba,EIN,Tax Year,Tax End Date,Location (Zipcode),Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,All Other Contributions,Noncash Contributions,Total
EIN_YEAR,,,,,,,,,,,,,
952648050_2020,SAN DIEGO YOUTH SERVICES,952648050,2020,06-30-2021,92110,0,0,0,0,19387646,322601,0,19710247
952648050_2019,SAN DIEGO YOUTH SERVICES,952648050,2019,06-30-2020,92110,0,0,0,0,19632420,203870,0,19836290
952648050_2018,SAN DIEGO YOUTH SERVICES,952648050,2018,06-30-2019,92110,0,0,0,0,19070850,144882,0,19215732
952648050_2017,SAN DIEGO YOUTH SERVICES,952648050,2017,06-30-2018,92110,0,0,0,0,17914417,225235,0,18139652
952648050_2016,SAN DIEGO YOUTH SERVICES,952648050,2016,06-30-2017,92110,0,0,0,0,0,339948,0,339948
...,...,...,...,...,...,...,...,...,...,...,...,...,...
331146733_2019,THINK DIGNITY,331146733,2019,12-31-2019,92104,0,0,48628,0,129001,125440,43807,303069
331146733_2018,THINK DIGNITY,331146733,2018,12-31-2018,92104,0,0,33682,0,114747,324226,25426,472655
331146733_2017,THINK DIGNITY,331146733,2017,12-31-2017,92104,0,4925,11158,0,106172,255771,0,381968


In [54]:
no_errorsEIN.to_csv('no_errorsEIN.csv')

#### Working with the data : Preparing Dataset

In [55]:
EINinfo = pd.read_csv('no_errorsEIN.csv')

In [56]:
np.unique(EINinfo.get('EIN'))

array([113723093, 204374795, 237161267, 237279074, 237332048, 237334012,
       261712580, 263405689, 270447059, 270865318, 272917644, 273390797,
       311750839, 330006089, 330008269, 330122462, 330210280, 330215585,
       330217339, 330317950, 330352148, 330433314, 330492304, 330496092,
       330497515, 330553621, 330553704, 330602842, 330618893, 330623634,
       330902617, 331029843, 331146733, 465055513, 550806460, 562613191,
       571162424, 571214920, 821946283, 822363154, 931008369, 941676390,
       942358632, 951644024, 951648184, 951729734, 951869806, 951874073,
       951944230, 951945256, 952039198, 952111196, 952157559, 952213995,
       952422704, 952457354, 952648050, 952653869, 952693142, 952794253,
       952833205, 952850121, 952877102, 952880426, 952949636, 953031682,
       953138268, 953140767, 953244085, 953248148, 953302967, 953315571,
       953368020, 953497926, 953649525, 953750738, 953782164, 953798088,
       953837714, 953950196, 956379598], dtype=int6

In [59]:
EINinfo = EINinfo.set_index('EIN_YEAR')#.drop('Unnamed: 0',axis=1)
EINinfo

,ba,EIN,Tax Year,Tax End Date,Location (Zipcode),Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,All Other Contributions,Noncash Contributions,Total
EIN_YEAR,,,,,,,,,,,,,
952648050_2020,SAN DIEGO YOUTH SERVICES,952648050,2020,06-30-2021,92110,0,0,0,0,19387646,322601,0,19710247
952648050_2019,SAN DIEGO YOUTH SERVICES,952648050,2019,06-30-2020,92110,0,0,0,0,19632420,203870,0,19836290
952648050_2018,SAN DIEGO YOUTH SERVICES,952648050,2018,06-30-2019,92110,0,0,0,0,19070850,144882,0,19215732
952648050_2017,SAN DIEGO YOUTH SERVICES,952648050,2017,06-30-2018,92110,0,0,0,0,17914417,225235,0,18139652
952648050_2016,SAN DIEGO YOUTH SERVICES,952648050,2016,06-30-2017,92110,0,0,0,0,0,339948,0,339948
...,...,...,...,...,...,...,...,...,...,...,...,...,...
331146733_2019,THINK DIGNITY,331146733,2019,12-31-2019,92104,0,0,48628,0,129001,125440,43807,303069
331146733_2018,THINK DIGNITY,331146733,2018,12-31-2018,92104,0,0,33682,0,114747,324226,25426,472655
331146733_2017,THINK DIGNITY,331146733,2017,12-31-2017,92104,0,4925,11158,0,106172,255771,0,381968


##### Grouped by Year

In [ ]:
EIN_Year = EINinfo.groupby('Tax Year').sum().drop(labels = ['ba','EIN','Location (Zipcode)'], axis=1)
EIN_Year.loc[2015.0, 'Fundraising Events'] -= 265175
EIN_Year.to_csv('EIN_Year.csv')

In [60]:
EIN_errs = find_errors(EINinfo)
EIN_errs

('550806460_2022',
 '953302967_2013',
 '952039198_2020',
 '952039198_2013',
 '953497926_2016',
 '272917644_2017',
 '951874073_2015',
 '263405689_2015',
 '953368020_2015',
 '571214920_2016',
 '331146733_2017')

#### Find errors, then prep data to double filter

In [61]:
errs1 =EINinfo[EINinfo.index.isin(EIN_errs)]

In [62]:
EINerrs = [int(err[:-5]) for err in EIN_errs]

#### Double filter testing

In [63]:
parsedErrEINs = grabber(EINerrs, errors=True)

0 / 11
1 / 11
2 / 11
3 / 11
4 / 11
5 / 11
6 / 11
7 / 11
8 / 11
9 / 11
10 / 11
11 / 11


In [64]:
no_errorsEIN = parsedErrEINs.dropna().reset_index(drop=True)
int_columns = no_errorsEIN.columns.difference(['ba','Tax End Date'])  # Skip the 'Tax End Date' column
no_errorsEIN[int_columns] = no_errorsEIN[int_columns].astype(int)
no_errorsEIN['EIN_YEAR'] = no_errorsEIN['EIN'].astype(str) + '_' + no_errorsEIN['Tax Year'].astype(str)
no_errorsEIN.set_index('EIN_YEAR')

,ba,EIN,Tax Year,Tax End Date,Location (Zipcode),Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,All Other Contributions,Noncash Contributions,Total
EIN_YEAR,,,,,,,,,,,,,
550806460_2022,Bilateral Safety Corridor Coalition,550806460,2022,12-31-2022,91950,0,0,8407,0,291440,270209,0,570056
550806460_2021,Bilateral Safety Corridor Coalition,550806460,2021,12-31-2021,91950,0,0,0,0,239763,227448,0,467211
550806460_2020,Bilateral Safety Corridor Coalition,550806460,2020,12-31-2020,91950,0,0,0,0,161068,218547,6000,379615
550806460_2019,Bilateral Safety Corridor Coalition,550806460,2019,12-31-2019,91950,0,0,0,0,0,269047,0,269047
550806460_2018,BILATERAL SAFETY CORRIDOR COALITION,550806460,2018,12-31-2018,91950,0,0,0,0,0,297479,0,297479
...,...,...,...,...,...,...,...,...,...,...,...,...,...
331146733_2019,THINK DIGNITY,331146733,2019,12-31-2019,92104,0,0,48628,0,129001,125440,43807,303069
331146733_2018,THINK DIGNITY,331146733,2018,12-31-2018,92104,0,0,33682,0,114747,324226,25426,472655
331146733_2017,THINK DIGNITY,331146733,2017,12-31-2017,92104,0,4925,15100,0,106172,255771,0,381968


In [66]:
no_errorsEIN.to_csv('no_errors2.csv')

In [69]:
EINErrinfo = pd.read_csv('no_errors2.csv')
EINErrinfo = EINErrinfo.set_index('EIN_YEAR')
EINErrinfo_index = list(EINErrinfo.index)
#EINinfo[EINinfo.index =='952039198_2021.0']

## EIN Dataset

In [38]:
def standardize_text(text):
    return re.sub(r'[{}\'‘’]'.format(string.punctuation), '', text).upper().strip().replace('\n',' ')

In [445]:
def replace_corrected_data_and_standardize(df1, df2, concat=False):
    to_std = ['Organization Name', 'EIN', 'Tax Year', 'Location (Zipcode)', 'Federate Campaigns', 'Membership Dues',
              'Fundraising Events', 'Related Organizations', 'Government Grants', 'All Other Contributions',
              'Noncash Contributions', 'Total']

    df1 = df1.reset_index()
    df2 = df2.reset_index()
    
    # Standardize selected columns (excluding 'Tax End Date') in both dataframes
    for col in to_std:
        if col != 'Tax End Date':
            df1[col] = df1[col].apply(lambda x: standardize_text(x) if isinstance(x, str) else x)
            df2[col] = df2[col].apply(lambda x: standardize_text(x) if isinstance(x, str) else x)

    # Replace errors with corrected information
    for index_value, corrected_row in df2.iterrows():
        ein_year = corrected_row['EIN_YEAR']
        matching_rows = df1['EIN_YEAR'] == ein_year
        for column in df1.columns:
            df1.loc[matching_rows, column] = corrected_row[column]

    if not concat:
        combined_df = df1
    else:
        combined_df = pd.concat([df1, df2])
    
    # Calculate new column 'Philanthropic_Total'
    combined_df['Philanthropic_Total'] = combined_df['Total'] - combined_df['Government Grants']

    # Save to CSV
    combined_df.to_csv('EIN_dataset.csv', index=False)

    return combined_df


In [70]:
EINinfo_copy = EINinfo.copy()
EINErrinfo_copy = EINErrinfo.copy()

EINinfo_copy.rename(columns={'ba': 'Organization Name'}, inplace=True)
EINErrinfo_copy.rename(columns={'ba': 'Organization Name'}, inplace=True)

EIN_dataset_df = replace_corrected_data_and_standardize(EINinfo_copy, EINErrinfo_copy)

# Print the resulting DataFrame
EIN_dataset_df

,All Other Contributions,EIN,EIN_YEAR,Federate Campaigns,Fundraising Events,Government Grants,Location (Zipcode),Membership Dues,Noncash Contributions,Organization Name,Related Organizations,Tax End Date,Tax Year,Total,Unnamed: 0,Philanthropic_Total
0,270209,550806460,550806460_2022,0,8407,291440,91950,0,0,BILATERAL SAFETY CORRIDOR COALITION,0,12-31-2022,2022,570056,0.0,278616
1,227448,550806460,550806460_2021,0,0,239763,91950,0,0,BILATERAL SAFETY CORRIDOR COALITION,0,12-31-2021,2021,467211,1.0,227448
2,218547,550806460,550806460_2020,0,0,161068,91950,0,6000,BILATERAL SAFETY CORRIDOR COALITION,0,12-31-2020,2020,379615,2.0,218547
3,269047,550806460,550806460_2019,0,0,0,91950,0,0,BILATERAL SAFETY CORRIDOR COALITION,0,12-31-2019,2019,269047,3.0,269047
4,297479,550806460,550806460_2018,0,0,0,91950,0,0,BILATERAL SAFETY CORRIDOR COALITION,0,12-31-2018,2018,297479,4.0,297479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616,125440,331146733,331146733_2019,0,48628,129001,92104,0,43807,THINK DIGNITY,0,12-31-2019,2019,303069,NaN,174068
617,324226,331146733,331146733_2018,0,33682,114747,92104,0,25426,THINK DIGNITY,0,12-31-2018,2018,472655,NaN,357908
618,255771,331146733,331146733_2017,0,11158,106172,92104,4925,0,THINK DIGNITY,0,12-31-2017,2017,381968,NaN,275796
619,114551,331146733,331146733_2016,0,20231,121084,92104,3445,11272,THINK DIGNITY,0,12-31-2016,2016,259311,NaN,138227


##### Standardize EIN dataset

In [ ]:
EIN_dataset = EIN_dataset_df.to_csv('EIN_dataset_1.csv')
EIN_dataset_df

In [ ]:
np.unique(EINErrinfo.get('EIN'))

In [ ]:
EINErrinfo = EINErrinfo.set_index('EIN_YEAR')

In [ ]:
EINErr_errs = find_errors(EINErrinfo)
EINErr_errs

In [ ]:
errs2 = EINErrinfo[EINErrinfo.index.isin(EINErr_errs)]

In [ ]:
[err for err in EINErr_errs if err in EIN_errs]

In [ ]:
pd.merge(errs1, errs2, how='inner')

In [ ]:
einyearcsv = pd.read_csv('EIN_Year.csv')

In [ ]:
einyearcsv

### New EINs

In [71]:
new_eins = [204374795,
311750839,
330553704,
330498557,
880279721,
330352148,
330352148,
951729734,
237332048]

In [72]:
new_ein990 = grabber(new_eins)

0 / 9
1 / 9
2 / 9
3 / 9
4 / 9
5 / 9
6 / 9
7 / 9
8 / 9
9 / 9


In [ ]:
new_ein990[new_ein990.get('EIN') == 880279721, 330498557]

In [87]:
new_EINinfo_df = new_ein990.dropna().reset_index(drop=True)
int_columns = new_EINinfo_df.columns.difference(['ba','Tax End Date'])  # Skip the 'Tax End Date' column
new_EINinfo_df[int_columns] = new_EINinfo_df[int_columns].astype(int)
new_EINinfo_df['EIN_YEAR'] = new_EINinfo_df['EIN'].astype(str) + '_' + new_EINinfo_df['Tax Year'].astype(str)
new_EINinfo_df.set_index('EIN_YEAR')

,ba,EIN,Tax Year,Tax End Date,Location (Zipcode),Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,All Other Contributions,Noncash Contributions,Total
EIN_YEAR,,,,,,,,,,,,,
204374795_2021,JACOBS & CUSHMAN SAN DIEGO FOOD BANK,204374795,2021,06-30-2022,92121,0,0,349596,0,6327592,87887254,58312927,94564442
204374795_2020,JACOBS & CUSHMAN SAN DIEGO FOOD BANK,204374795,2020,06-30-2021,92121,0,0,251384,0,10838132,113522301,80913471,124611817
204374795_2019,JACOBS & CUSHMAN SAN DIEGO FOOD BANK,204374795,2019,06-30-2020,92121,0,0,103012,0,5257407,75773240,58281891,81133659
204374795_2018,JACOBS & CUSHMAN SAN DIEGO FOOD BANK,204374795,2018,06-30-2019,92121,0,0,316076,0,2593934,48722765,41159936,51632775
204374795_2017,JACOBS & CUSHMAN SAN DIEGO FOOD BANK,204374795,2017,06-30-2018,92121,0,0,191553,0,2106601,41309129,35087496,43607283
204374795_2016,JACOBS & CUSHMAN SAN DIEGO FOOD BANK,204374795,2016,06-30-2017,92121,0,0,154260,0,1623355,38415684,31883433,40193299
204374795_2015,JACOBS & CUSHMAN SAN DIEGO FOOD BANK,204374795,2015,06-30-2016,92121,0,0,117922,0,2175671,31097166,25686164,33390759
204374795_2014,JACOBS & CUSHMAN SAN DIEGO FOOD BANK,204374795,2014,06-30-2015,92121,0,0,143289,0,8567254,24065867,25032864,32776410
204374795_2013,JACOBS & CUSHMAN SAN DIEGO FOOD BANK,204374795,2013,06-30-2014,0,0,0,131556,0,2230566,30090121,25346991,32452243


In [88]:
ein_data = pd.read_csv('EIN_dataset.csv')
ein_data =ein_data.set_index('EIN_YEAR')

#### DOES NOT INCLUDE MISSED MANUAL PULL INFO

In [90]:
# Create copies of the DataFrames and rename columns in the copies
EIN_dataset_df_copy = ein_data.copy()
EIN_dataset_df_copy.rename(columns={'ba': 'Organization Name'}, inplace=True)

new_EINinfo_copy = new_EINinfo_df.reset_index().copy()
new_EINinfo_copy.rename(columns={'ba': 'Organization Name'}, inplace=True)

# Call the function with the modified copies
new_concat_EINinfo_df = replace_corrected_data_and_standardize(EIN_dataset_df_copy, new_EINinfo_copy, concat=True).set_index('EIN_YEAR')\
                    .drop(['index','level_0','Unnamed: 0'],axis=1).drop_duplicates()

# Print the resulting DataFrame
new_concat_EINinfo_df.to_csv('Phil_Dataset_1.csv')

In [91]:
new_concat_EINinfo_df

,All Other Contributions,EIN,Federate Campaigns,Fundraising Events,Government Grants,Location (Zipcode),Membership Dues,Noncash Contributions,Organization Name,Related Organizations,Tax End Date,Tax Year,Total,Philanthropic_Total
EIN_YEAR,,,,,,,,,,,,,,
550806460_2022,270209,550806460,0,8407,291440,91950,0,0,BILATERAL SAFETY CORRIDOR COALITION,0,12-31-2022,2022,570056,278616
550806460_2021,227448,550806460,0,0,239763,91950,0,0,BILATERAL SAFETY CORRIDOR COALITION,0,12-31-2021,2021,467211,227448
550806460_2020,218547,550806460,0,0,161068,91950,0,6000,BILATERAL SAFETY CORRIDOR COALITION,0,12-31-2020,2020,379615,218547
550806460_2019,269047,550806460,0,0,0,91950,0,0,BILATERAL SAFETY CORRIDOR COALITION,0,12-31-2019,2019,269047,269047
550806460_2018,297479,550806460,0,0,0,91950,0,0,BILATERAL SAFETY CORRIDOR COALITION,0,12-31-2018,2018,297479,297479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204374795_2017,41309129,204374795,0,191553,2106601,92121,0,35087496,JACOBS CUSHMAN SAN DIEGO FOOD BANK,0,06-30-2018,2017,43607283,41500682
204374795_2016,38415684,204374795,0,154260,1623355,92121,0,31883433,JACOBS CUSHMAN SAN DIEGO FOOD BANK,0,06-30-2017,2016,40193299,38569944
204374795_2015,31097166,204374795,0,117922,2175671,92121,0,25686164,JACOBS CUSHMAN SAN DIEGO FOOD BANK,0,06-30-2016,2015,33390759,31215088


In [92]:
new_EINinfo = new_EINinfo_df.to_csv('new_EINinfo')


In [94]:
new_EINerrs = find_errors(new_EINinfo_df)
new_EINerrs = np.unique([int(err[:-5]) for err in new_EINerrs])
#newErrEINs = grabber(new_EINerrs, errors=True)
#EIN_dataset_df.loc[new_eins]#, new_EINinfo_df

array([], dtype=float64)

In [79]:
new_ErrEINs = newErrEINs.dropna().reset_index(drop=True)
new_ErrEINs.loc[:, 'EIN':'Total'] = new_ErrEINs.loc[:, 'EIN':'Total'].astype(int)
new_ErrEINs.loc[:, 'EIN_YEAR'] = new_ErrEINs['EIN'].astype(str) + '_' + new_ErrEINs['Tax Year'].astype(str)
new_ErrEINs = new_ErrEINs.set_index('EIN_YEAR')
new_ErrEINs

KeyError: 'EIN'

In [85]:
desired_order = ['Organization Name','EIN','Tax Year','Tax End Date','Location (Zipcode)','Federate Campaigns','Membership Dues','Fundraising Events','Related Organizations','Government Grants','All Other Contributions','Noncash Contributions','Total', 'Philanthropic_Total']

In [86]:
# Create copies of the DataFrames and rename columns in the copies
new_ErrEINs_copy = new_ErrEINs.copy()
new_ErrEINs_copy.rename(columns={'ba': 'Organization Name'}, inplace=True)

#EIN_dataset_df_copy = EIN_dataset_df.copy()
#EIN_dataset_df_copy.rename(columns={'ba': 'Organization Name'}, inplace=True)

# Call the function with the modified copies
cor_EINinfo_df = replace_corrected_data_and_standardize(EIN_dataset_df, new_ErrEINs_copy)

# Print the resulting DataFrame
final_EINinfo_df = cor_EINinfo_df.set_index('EIN_YEAR').drop('index',axis=1).drop_duplicates()
final_EINinfo_df = final_EINinfo_df[desired_order]
final_EINinfo_df

KeyError: "None of [Index(['Organization Name', 'EIN', 'Tax Year', 'Location (Zipcode)',\n       'Federate Campaigns', 'Membership Dues', 'Fundraising Events',\n       'Related Organizations', 'Government Grants', 'All Other Contributions',\n       'Noncash Contributions', 'Total'],\n      dtype='object')] are in the [columns]"

In [ ]:
#final_EINinfo_df.to_csv('Phil_Dataset.csv')

In [292]:
phil_df = pd.read_csv('Phil_Dataset_1.csv') 
phil_df = phil_df.set_index('EIN_YEAR')
phil_df

,All Other Contributions,EIN,Federate Campaigns,Fundraising Events,Government Grants,Location (Zipcode),Membership Dues,Noncash Contributions,Organization Name,Related Organizations,Tax End Date,Tax Year,Total,Philanthropic_Total
EIN_YEAR,,,,,,,,,,,,,,
550806460_2022,270209,550806460,0,8407,291440,91950,0,0,BILATERAL SAFETY CORRIDOR COALITION,0,12-31-2022,2022,570056,278616
550806460_2021,227448,550806460,0,0,239763,91950,0,0,BILATERAL SAFETY CORRIDOR COALITION,0,12-31-2021,2021,467211,227448
550806460_2020,218547,550806460,0,0,161068,91950,0,6000,BILATERAL SAFETY CORRIDOR COALITION,0,12-31-2020,2020,379615,218547
550806460_2019,269047,550806460,0,0,0,91950,0,0,BILATERAL SAFETY CORRIDOR COALITION,0,12-31-2019,2019,269047,269047
550806460_2018,297479,550806460,0,0,0,91950,0,0,BILATERAL SAFETY CORRIDOR COALITION,0,12-31-2018,2018,297479,297479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204374795_2017,41309129,204374795,0,191553,2106601,92121,0,35087496,JACOBS CUSHMAN SAN DIEGO FOOD BANK,0,06-30-2018,2017,43607283,41500682
204374795_2016,38415684,204374795,0,154260,1623355,92121,0,31883433,JACOBS CUSHMAN SAN DIEGO FOOD BANK,0,06-30-2017,2016,40193299,38569944
204374795_2015,31097166,204374795,0,117922,2175671,92121,0,25686164,JACOBS CUSHMAN SAN DIEGO FOOD BANK,0,06-30-2016,2015,33390759,31215088


## Getting Manual Pull Rows that aren't included in the Scraped Data

In [293]:
full_phil = pd.read_csv('Phil Dataset.csv')

In [218]:
raw_manual_pull = pd.read_csv('990 Manual Pull - Tax End Year.csv')
#raw_manual_pull = raw_manual_pull.set_index('EIN_YEAR')
raw_manual_pull

,ba,EIN,Tax Year,Tax End Date,Location (Zipcode),Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,"All Other Contributions gifts, grants and similar amounts not included above",Noncash constributions included in lines 1a - 1f,Total,Type,Unnamed: 14,Unnamed: 15,Unnamed: 16,EIN_YEAR
0,Regional Task Force on the Homeless,113723093.0,2015.0,NaN,92123.0,0,0,0,0,1110060,47330,0,1157390,A,1157390.0,True,0.0,113723093_2015
1,Regional Task Force on the Homeless,113723093.0,2016.0,NaN,92123.0,0,6677,0,0,1922156,65214,0,1994047,A,1994047.0,True,0.0,113723093_2016
2,Regional Task Force on the Homeless,113723093.0,2018.0,NaN,92123.0,0,0,0,0,2505451,865020,0,3370471,A,3370471.0,True,0.0,113723093_2018
3,Regional Task Force on the Homeless,113723093.0,2019.0,NaN,92123.0,0,0,0,0,9344677,182239,0,9526916,A,9526916.0,True,0.0,113723093_2019
4,Regional Task Force on the Homeless,113723093.0,2020.0,NaN,92123.0,0,0,0,0,17841834,183535,0,18025369,A,18025369.0,True,0.0,113723093_2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,Operation Hope North County,571214920.0,2020.0,NaN,92084.0,0,0,0,0,422615,692114,0,1114729,NaN,1114729.0,True,NaN,571214920_2020
347,Kingdom Causes dba City Net,571162424.0,2021.0,NaN,90807.0,89250,0,0,0,429733,790394,27870,1309377,NaN,1309377.0,True,NaN,571162424_2021
348,NaN,NaN,NaN,NaN,NaN,"$35,571,906.00","$47,967.00","$19,229,327.00","$23,349,134.00","$3,384,170,346.00","$759,399,657.00","$218,330,487.00","$4,230,190,126.00",NaN,NaN,NaN,NaN,NaN
349,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$846,019,780.00",NaN,NaN,NaN,NaN,NaN


In [227]:
manual_pull = raw_manual_pull[raw_manual_pull.get('Type') == 'A'].dropna(thresh=1).reset_index(drop=True).drop(['Type'],axis=1)
#manual_pull.loc[:, 'EIN':'Total'] = manual_pull.loc[:, 'EIN':'Total'].astype(int)
#manual_pull.loc[:, 'EIN_YEAR'] = manual_pull['EIN'].astype(str) + '_' + manual_pull['Tax Year'].astype(str)
manual_pull = manual_pull.set_index('EIN_YEAR')
#raw_manual_pull[~raw_manual_pull.index.isin(manual_pull.index)]
manual_pull

,ba,EIN,Tax Year,Tax End Date,Location (Zipcode),Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,"All Other Contributions gifts, grants and similar amounts not included above",Noncash constributions included in lines 1a - 1f,Total,Unnamed: 14,Unnamed: 15,Unnamed: 16
EIN_YEAR,,,,,,,,,,,,,,,,
113723093_2015,Regional Task Force on the Homeless,113723093.0,2015.0,NaN,92123.0,0,0,0,0,1110060,47330,0,1157390,1157390.0,True,0.0
113723093_2016,Regional Task Force on the Homeless,113723093.0,2016.0,NaN,92123.0,0,6677,0,0,1922156,65214,0,1994047,1994047.0,True,0.0
113723093_2018,Regional Task Force on the Homeless,113723093.0,2018.0,NaN,92123.0,0,0,0,0,2505451,865020,0,3370471,3370471.0,True,0.0
113723093_2019,Regional Task Force on the Homeless,113723093.0,2019.0,NaN,92123.0,0,0,0,0,9344677,182239,0,9526916,9526916.0,True,0.0
113723093_2020,Regional Task Force on the Homeless,113723093.0,2020.0,NaN,92123.0,0,0,0,0,17841834,183535,0,18025369,18025369.0,True,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270447059_2018,Dreams for Change Inc.,270447059.0,2018.0,NaN,92176.0,0,0,0,0,345041,492397,0,837438,837438.0,True,NaN
270447059_2019,Dreams for Change Inc.,270447059.0,2019.0,NaN,92176.0,0,0,0,0,600504,421536,0,1022040,1022040.0,True,NaN
270447059_2020,Dreams for Change Inc.,270447059.0,2020.0,NaN,92176.0,0,0,0,0,717584,434903,0,1152487,1152487.0,True,NaN


In [221]:
#new_index = [index[:-5] for index in phil_df.index]
#phil_df.reset_index()
#phil_df.index = new_index
full_phil = full_phil.drop(['Unnamed: 0'],axis=1)

In [228]:
pull_these = list(manual_pull.index.difference(phil_df.index))
pull_these

['330006089_2015',
 '330217339_2015',
 '330217339_2016',
 '330217339_2017',
 '330217339_2018',
 '330217339_2019',
 '330433314_2015',
 '330433314_2016',
 '330433314_2017',
 '330433314_2018',
 '330492304_2015',
 '330492304_2016',
 '330492304_2017',
 '330492304_2018',
 '330492304_2019',
 '330602842_2015',
 '330602842_2016',
 '330602842_2017',
 '330602842_2018',
 '330602842_2019',
 '330618893_2016',
 '330618893_2018',
 '331029843_2015',
 '331029843_2017',
 '331029843_2018',
 '331029843_2019',
 '465055513_2016',
 '465055513_2017',
 '562613191_2015',
 '562613191_2016',
 '822363154_2019',
 '941676390_2016',
 '941676390_2017',
 '941676390_2018',
 '941676390_2019',
 '952111196_2015',
 '952111196_2016',
 '952111196_2017',
 '952111196_2018',
 '952648050_2015',
 '952648050_2016',
 '952648050_2017',
 '952648050_2018',
 '952648050_2019',
 '952850121_2015',
 '952850121_2016',
 '953244085_2015',
 '953244085_2016',
 '953244085_2017',
 '953244085_2018',
 '953244085_2019',
 '953649525_2015',
 '953649525_

In [229]:
to_pull = np.unique([ein[:-5] for ein in pull_these])
to_pull

array(['330006089', '330217339', '330433314', '330492304', '330602842',
       '330618893', '331029843', '465055513', '562613191', '822363154',
       '941676390', '952111196', '952648050', '952850121', '953244085',
       '953649525', '953750738'], dtype='<U9')

In [234]:
not_scraped = manual_pull.loc[pull_these]
not_scraped['EIN'] = [int(ein) for ein in not_scraped['EIN']]
not_scraped['Tax Year'] = [int(year) for year in not_scraped['Tax Year']]
not_scraped['Location (Zipcode)'] = [int(loc) for loc in not_scraped['Location (Zipcode)']]
not_scraped = not_scraped.drop(['Unnamed: 14','Unnamed: 15','Unnamed: 16'],axis=1)

In [225]:
to_std = ['Organization Name', 'EIN', 'Tax Year', 'Tax End Date','Location (Zipcode)', 'Federate Campaigns', 'Membership Dues',
              'Fundraising Events', 'Related Organizations', 'Government Grants', 'All Other Contributions',
              'Noncash Contributions', 'Total']

In [235]:
not_scraped.columns = ['Organization Name', 'EIN', 'Tax Year', 'Tax End Date','Location (Zipcode)', 'Federate Campaigns', 'Membership Dues', \
                       'Fundraising Events', 'Related Organizations', 'Government Grants', 'All Other Contributions', \
                       'Noncash Contributions', 'Total']

int_columns = not_scraped.columns.difference(['Organization Name','EIN_YEAR', 'Tax End Date'])  # Skip the 'Tax End Date' column
not_scraped[int_columns] = not_scraped[int_columns].apply(pd.to_numeric, errors='coerce')
not_scraped = not_scraped.reset_index()

In [284]:
phil_df_copy = phil_df.copy().reset_index()
# Create an empty DataFrame to store the result
result_df = pd.DataFrame(columns=phil_df.columns)

# Iterate through each index value in the pull_these list
for index_value in pull_these:
    try:
        # Extract the organization EIN and year from the index value
        ein, year = index_value.split('_')
        
        # Create a mask to select the row with the same EIN and year from add_not_scraped
        mask = (not_scraped['EIN'] == float(ein)) & (not_scraped['Tax Year'] == float(year))
        
        # Get the corresponding row from add_not_scraped
        row_to_add = not_scraped[mask]
        
        # Append the row to the result DataFrame
        result_df = pd.concat([result_df, row_to_add])
    except Exception as e:
        print(f"An error occurred for {index_value}: {e}")

# Concatenate the original phil_df and the result_df
full_phil_df = pd.concat([phil_df_copy, result_df]).set_index('EIN_YEAR')
full_phil_df = replace_corrected_data_and_standardize(not_scraped, phil_df, concat=True)
full_phil_df = full_phil_df.drop(['index'],axis=1)

In [281]:
#full_phil_df.to_csv('Phil Dataset.csv')

In [409]:
full_phil_df=full_phil_df.set_index('EIN_YEAR')
full_phil = full_phil.set_index('EIN_YEAR')

KeyError: "None of ['EIN_YEAR'] are in the columns"

In [295]:
full_phil['EIN'] = [(str(ein))[:-2] for ein in full_phil['EIN']]
full_phil['Tax Year'] = [(str(year))[:-2] for year in full_phil['Tax Year']]

full_phil.loc[:, 'EIN_YEAR'] = full_phil['EIN'].astype(str) + '_' + full_phil['Tax Year'].astype(str)
full_phil

,Unnamed: 0,Organization Name,EIN,Tax Year,Location (Zipcode),Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,All Other Contributions,Noncash Contributions,Total,Philanthropic_Total,EIN_YEAR
0,113723093_2014,REGIONAL TASK FORCE ON THE HOMELESS INC,113723093,2014,92123.0,0,0,0,0,991765,94796,0,1086561,94796,113723093_2014
1,113723093_2015,REGIONAL TASK FORCE ON THE HOMELESS,113723093,2015,92123.0,0,0,0,0,1110060,47330,0,1157390,47330,113723093_2015
2,113723093_2016,REGIONAL TASK FORCE ON THE HOMELESS,113723093,2016,92123.0,0,6677,0,0,1922156,65214,0,1994047,71891,113723093_2016
3,113723093_2017,REGIONAL TASK FORCE ON THE HOMELESS,113723093,2017,92123.0,0,0,0,0,2021667,738875,0,2760542,738875,113723093_2017
4,113723093_2018,REGIONAL TASK FORCE ON THE HOMELESS,113723093,2018,92123.0,0,0,0,0,2505451,865020,0,3370471,865020,113723093_2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,941676390_2018,CALIFORNIA INDIAN LEGAL SERVICES INC,941676390,2018,92025.0,0,0,0,0,1819928,73318,0,1893246,73318,941676390_2018
638,941676390_2019,CALIFORNIA INDIAN LEGAL SERVICES INC,941676390,2019,92025.0,0,0,0,0,1765110,174234,0,1939344,174234,941676390_2019
639,952111196_2015,ST PAULS EPISCOPAL HOME INC,952111196,2015,92013.0,0,0,0,2297041,590897,894547,884493,3782485,3191588,952111196_2015
640,952850121_2015,SERVING SENIORS,952850121,2015,92101.0,0,0,0,0,3119195,1727053,96540,4846248,1727053,952850121_2015


In [308]:
desired_order = ['Organization Name','EIN','Tax Year','Tax End Date','Location (Zipcode)','Federate Campaigns','Membership Dues','Fundraising Events','Related Organizations','Government Grants','All Other Contributions','Noncash Contributions','Total', 'Philanthropic_Total']

In [309]:
new_phil_df = replace_corrected_data_and_standardize(full_phil, full_phil_df).set_index('EIN_YEAR')
new_phil_df = new_phil_df[desired_order]

In [318]:
new_phil_df[new_phil_df['Tax End Date'].isna()]

,Organization Name,EIN,Tax Year,Tax End Date,Location (Zipcode),Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,All Other Contributions,Noncash Contributions,Total,Philanthropic_Total
EIN_YEAR,,,,,,,,,,,,,,
237332048_2018,THE SAN DIEGO LESBIANGAY BISEXUAL TRANSGENDER ...,237332048,2018,NaN,92163.0,0,0,0,0,2417854,4898662,0,7316516,4898662
237332048_2017,THE SAN DIEGO LESBIANGAY BISEXUAL TRANSGENDER ...,237332048,2017,NaN,92163.0,0,0,0,0,2618909,4786412,0,7405321,4786412
237332048_2016,THE SAN DIEGO LESBIANGAY BISEXUAL TRANSGENDER ...,237332048,2016,NaN,92163.0,0,0,0,0,2674347,4105894,0,6780241,4105894
237332048_2015,THE SAN DIEGO LESBIANGAY BISEXUAL TRANSGENDER ...,237332048,2015,NaN,92163.0,0,0,0,0,2265389,2588997,0,4854386,2588997
237332048_2014,THE SAN DIEGO LESBIANGAY BISEXUAL TRANSGENDER ...,237332048,2014,NaN,92163.0,0,0,0,0,1807212,2867228,0,4674440,2867228
330006089_2015,DEAF COMMUNITY SERVICES OF SAN DIEGO,330006089,2015,NaN,92108.0,3270,0,0,0,1329774,61532,0,1394576,64802
330433314_2015,NATIONAL CONFLICT RESOLUTION CENTER,330433314,2015,NaN,92101.0,0,0,0,0,851714,1076360,0,1928074,1076360
330433314_2016,NATIONAL CONFLICT RESOLUTION CENTER,330433314,2016,NaN,92101.0,0,0,0,0,806892,1300903,0,2107795,1300903
330433314_2017,NATIONAL CONFLICT RESOLUTION CENTER,330433314,2017,NaN,92101.0,0,0,0,0,1514007,1231820,0,2745827,1231820


In [321]:
new_phil_df.to_csv('Phil Data - Tax Date.csv')

In [326]:
errs = find_errors(new_phil_df)
errs = [err[:-5] for err in errs]
fix_errs = grabber(errs, errors=True)

0 / 10
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10


In [378]:
fixing = fix_errs.dropna().reset_index(drop=True)
int_columns = fix_errs.columns.difference(['ba','Tax End Date'])  # Skip the 'Tax End Date' column
fixing[int_columns] = fixing[int_columns].astype(int)
fixing['EIN_YEAR'] = fixing['EIN'].astype(str) + '_' + fixing['Tax Year'].astype(str)
fixing = fixing.set_index('EIN_YEAR')
fixing.rename(columns={'ba': 'Organization Name'}, inplace=True)

In [415]:
fixing.loc['951874073_2015', 'Fundraising Events'] = 0
find_errors(fixing)
#fixing[fixing.index == '951874073_2015']
new_phil_df = new_phil_df.set_index('EIN_YEAR')#.drop('index', axis=1)
fixing = fixing.set_index('EIN_YEAR')

KeyError: "None of ['EIN_YEAR'] are in the columns"

In [446]:
fixing_phil_df = replace_corrected_data_and_standardize(new_phil_df, fixing).set_index('EIN_YEAR')
fixing_phil_df = fixing_phil_df[desired_order]
#fixing_phil_df[fixing_phil_df.index == '951874073_2015']

In [447]:
find_errors(fixing_phil_df)

()

In [427]:
new_phil_df = new_phil_df.reset_index().drop('Philanthropic_Total',axis=1)
fixing = fixing.reset_index()

In [454]:
fixing_phil_df.to_csv('Phil Data - Tax Date.csv')

In [441]:
#fixing = fixing.drop(['level_0','index'],axis=1)
new_phil_df = new_phil_df.drop('Philanthropic_Total',axis=1)

In [453]:
fixing_phil_df['Tax End Date'] = fixing_phil_df['Tax End Date'].astype(str).apply(lambda x: "{}-{}-{}".format(x[:-6], x[-6:-4], x[-4:]))

(fixing_phil_df)

,Organization Name,EIN,Tax Year,Tax End Date,Location (Zipcode),Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,All Other Contributions,Noncash Contributions,Total,Philanthropic_Total
EIN_YEAR,,,,,,,,,,,,,,
330006089_2015,DEAF COMMUNITY SERVICES OF SAN DIEGO,330006089.0,2015.0,6-30-2016,92108.0,3270,0,0,0,1329774,61532,0,1394576,64802
330217339_2015,CRISIS HOUSE,330217339.0,2015.0,6-30-2016,92020.0,0,0,0,0,756702,273265,169610,1029967,273265
330217339_2016,CRISIS HOUSE,330217339.0,2016.0,6-30-2017,92020.0,0,0,0,0,802795,342525,189186,1145320,342525
330217339_2017,CRISIS HOUSE,330217339.0,2017.0,6-30-2018,92020.0,0,0,0,0,1433825,346726,183702,1780551,346726
330217339_2018,CRISIS HOUSE,330217339.0,2018.0,6-30-2019,92020.0,0,0,10770,0,1424698,512193,118107,1947661,522963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941676390_2018,CALIFORNIA INDIAN LEGAL SERVICES INC,941676390,2018,--nan,92025.0,0,0,0,0,1819928,73318,0,1893246,73318
941676390_2019,CALIFORNIA INDIAN LEGAL SERVICES INC,941676390,2019,--nan,92025.0,0,0,0,0,1765110,174234,0,1939344,174234
952111196_2015,ST PAULS EPISCOPAL HOME INC,952111196,2015,--nan,92013.0,0,0,0,2297041,590897,894547,884493,3782485,3191588


## Parsing dataset by month

In [33]:
data = pd.read_csv('Phil Dataset - Tax Date.csv')

In [3]:
def expand_rows(row):
    expanded_rows = []

    tax_end_date = pd.to_datetime(row['Tax End Date'])
    amount_per_month = row['Philanthropic_Total'] / 12

    for _ in range(1, 13):
        tax_end_date -= pd.DateOffset(months=1)
        expanded_rows.append({
            "EIN_YEAR": row['EIN_YEAR'],
            "Tax End Date": tax_end_date,
            "Philanthropic_Total": amount_per_month
        })

    expanded_df = pd.DataFrame(expanded_rows)
    return expanded_df

In [9]:
data = pd.read_csv('test.csv')
data

,EIN_YEAR,Organization Name,EIN,Tax Year,Tax End Date,Location (Zipcode),Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,All Other Contributions,Noncash Contributions,Total,Philanthropic_Total
0,237161267_2013,SOUTHERN CALIFORNIA TRIBAL CHAIRMENS ASSOCIATION,237161267,2013,12-31-2013,0,0,0,0,0,1.232018e+06,231853.5,11376.33333,1.463872e+06,231853.5
1,953837714_2014,INTERFAITH COMMUNITY SERVICES INC,953837714,2014,06-30-2015,92025,3029,0,0,0,0.000000e+00,4472449.0,665579.00000,4.475478e+06,4475478.0
2,953782164_2018,SOUTHERN INDIAN HEALTH COUNCIL,953782164,2018,09-30-2019,91901,0,0,0,0,1.282161e+07,171201.0,44396.00000,1.299282e+07,171201.0


In [47]:
# Convert 'Tax End Date' column to datetime format
data['Tax End Date'] = pd.to_datetime(data['Tax End Date'])

# Function to distribute values across months within a fiscal year
def distribute_values_fiscal_year(row):
    # Calculate fiscal year period based on the given end date
    fiscal_year_start_date = pd.Timestamp(year=row['Tax End Date'].year - 1, month=row['Tax End Date'].month, day=row['Tax End Date'].day) + pd.DateOffset(days=1)
    fiscal_year_end_date = fiscal_year_start_date + pd.DateOffset(years=1, days=-1)

    # Calculate the number of months in the fiscal year
    num_months = (fiscal_year_end_date.year - fiscal_year_start_date.year) * 12 + fiscal_year_end_date.month - fiscal_year_start_date.month + 1

    # Calculate the values to distribute (including the last two columns)
    values_to_distribute = row[7:] / num_months

    # Generate the fiscal year date range
    date_range = pd.date_range(start=fiscal_year_start_date, end=fiscal_year_end_date, freq='M')

    new_rows = []
    for date in date_range:
        new_row = row.copy()
        new_row['Tax End Date'] = date.strftime('%m-%d-%Y')
        new_row[7:] = values_to_distribute  # Update all columns from 7th column onwards
        new_rows.append(new_row)

    return new_rows

# Apply the function to each row and expand the DataFrame
expanded_data_fiscal_year = data.apply(distribute_values_fiscal_year, axis=1)
expanded_data_fiscal_year = pd.DataFrame([item for sublist in expanded_data_fiscal_year for item in sublist])


# Save the expanded data to a new CSV file
expanded_data_fiscal_year = expanded_data_fiscal_year.drop_duplicates()
expanded_data_fiscal_year.to_csv('Phil Dataset - Analysis.csv')

In [82]:
cc = pd.read_csv('Philanthropic Dataset - 20230808 - Analysis.csv')
cc.iloc[:, :-13]


,EIN_YEAR,Organization Name,EIN,Tax Year,Tax End Date,Location (Zipcode),Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,All Other Contributions,Noncash Contributions,Total,Philanthropic_Total
0,113723093_2014,REGIONAL TASK FORCE ON THE HOMELESS INC,113723093,2014,06-30-2015,92123.0,0.0,0.0,0.00000,0.0,82647.08333,7899.666667,0.000000,90546.75,7899.666667
1,113723093_2014,REGIONAL TASK FORCE ON THE HOMELESS INC,113723093,2014,05-30-2015,NaN,0.0,0.0,0.00000,0.0,82647.08333,7899.666667,0.000000,90546.75,7899.666667
2,113723093_2014,REGIONAL TASK FORCE ON THE HOMELESS INC,113723093,2014,04-30-2015,NaN,0.0,0.0,0.00000,0.0,82647.08333,7899.666667,0.000000,90546.75,7899.666667
3,113723093_2014,REGIONAL TASK FORCE ON THE HOMELESS INC,113723093,2014,03-30-2015,NaN,0.0,0.0,0.00000,0.0,82647.08333,7899.666667,0.000000,90546.75,7899.666667
4,113723093_2014,REGIONAL TASK FORCE ON THE HOMELESS INC,113723093,2014,02-28-2015,NaN,0.0,0.0,0.00000,0.0,82647.08333,7899.666667,0.000000,90546.75,7899.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7278,956379598_2021,CENTER FOR COMMUNITY SOLUTIONS,956379598,2021,11-30-2021,92109.0,0.0,0.0,38652.58333,0.0,459745.50000,70906.416670,1354.666667,569304.50,109559.000000
7279,956379598_2021,CENTER FOR COMMUNITY SOLUTIONS,956379598,2021,10-30-2021,92109.0,0.0,0.0,38652.58333,0.0,459745.50000,70906.416670,1354.666667,569304.50,109559.000000
7280,956379598_2021,CENTER FOR COMMUNITY SOLUTIONS,956379598,2021,09-30-2021,92109.0,0.0,0.0,38652.58333,0.0,459745.50000,70906.416670,1354.666667,569304.50,109559.000000
7281,956379598_2021,CENTER FOR COMMUNITY SOLUTIONS,956379598,2021,08-30-2021,92109.0,0.0,0.0,38652.58333,0.0,459745.50000,70906.416670,1354.666667,569304.50,109559.000000


In [83]:
manual_expanded_data = cc.iloc[:, :-13]

generated_expanded_data = expanded_data_fiscal_year
generated_expanded_data#.set_index('EIN_YEAR',inplace=True)

,Organization Name,EIN,Tax Year,Tax End Date,Location (Zipcode),Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,All Other Contributions,Noncash Contributions,Total,Philanthropic_Total
EIN_YEAR,,,,,,,,,,,,,,
330006089_2015,DEAF COMMUNITY SERVICES OF SAN DIEGO,330006089,2015,07-31-2015,92108,3270,0.0,0.0,0.0,110814.5,5.127667e+03,0.00,1.162147e+05,5.400167e+03
330006089_2015,DEAF COMMUNITY SERVICES OF SAN DIEGO,330006089,2015,08-31-2015,92108,3270,0.0,0.0,0.0,110814.5,5.127667e+03,0.00,1.162147e+05,5.400167e+03
330006089_2015,DEAF COMMUNITY SERVICES OF SAN DIEGO,330006089,2015,09-30-2015,92108,3270,0.0,0.0,0.0,110814.5,5.127667e+03,0.00,1.162147e+05,5.400167e+03
330006089_2015,DEAF COMMUNITY SERVICES OF SAN DIEGO,330006089,2015,10-31-2015,92108,3270,0.0,0.0,0.0,110814.5,5.127667e+03,0.00,1.162147e+05,5.400167e+03
330006089_2015,DEAF COMMUNITY SERVICES OF SAN DIEGO,330006089,2015,11-30-2015,92108,3270,0.0,0.0,0.0,110814.5,5.127667e+03,0.00,1.162147e+05,5.400167e+03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204374795_2013,JACOBS CUSHMAN SAN DIEGO FOOD BANK,204374795,2013,02-28-2014,0,0,0.0,10963.0,0.0,185880.5,2.507510e+06,2112249.25,2.704354e+06,2.518473e+06
204374795_2013,JACOBS CUSHMAN SAN DIEGO FOOD BANK,204374795,2013,03-31-2014,0,0,0.0,10963.0,0.0,185880.5,2.507510e+06,2112249.25,2.704354e+06,2.518473e+06
204374795_2013,JACOBS CUSHMAN SAN DIEGO FOOD BANK,204374795,2013,04-30-2014,0,0,0.0,10963.0,0.0,185880.5,2.507510e+06,2112249.25,2.704354e+06,2.518473e+06


In [84]:
manual_expanded_data.set_index('EIN_YEAR', inplace=True)

In [86]:
manual_expanded_data

,Organization Name,EIN,Tax Year,Tax End Date,Location (Zipcode),Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,All Other Contributions,Noncash Contributions,Total,Philanthropic_Total
EIN_YEAR,,,,,,,,,,,,,,
113723093_2014,REGIONAL TASK FORCE ON THE HOMELESS INC,113723093,2014,06-30-2015,92123.0,0.0,0.0,0.00000,0.0,82647.08333,7899.666667,0.000000,90546.75,7899.666667
113723093_2014,REGIONAL TASK FORCE ON THE HOMELESS INC,113723093,2014,05-30-2015,NaN,0.0,0.0,0.00000,0.0,82647.08333,7899.666667,0.000000,90546.75,7899.666667
113723093_2014,REGIONAL TASK FORCE ON THE HOMELESS INC,113723093,2014,04-30-2015,NaN,0.0,0.0,0.00000,0.0,82647.08333,7899.666667,0.000000,90546.75,7899.666667
113723093_2014,REGIONAL TASK FORCE ON THE HOMELESS INC,113723093,2014,03-30-2015,NaN,0.0,0.0,0.00000,0.0,82647.08333,7899.666667,0.000000,90546.75,7899.666667
113723093_2014,REGIONAL TASK FORCE ON THE HOMELESS INC,113723093,2014,02-28-2015,NaN,0.0,0.0,0.00000,0.0,82647.08333,7899.666667,0.000000,90546.75,7899.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
956379598_2021,CENTER FOR COMMUNITY SOLUTIONS,956379598,2021,11-30-2021,92109.0,0.0,0.0,38652.58333,0.0,459745.50000,70906.416670,1354.666667,569304.50,109559.000000
956379598_2021,CENTER FOR COMMUNITY SOLUTIONS,956379598,2021,10-30-2021,92109.0,0.0,0.0,38652.58333,0.0,459745.50000,70906.416670,1354.666667,569304.50,109559.000000
956379598_2021,CENTER FOR COMMUNITY SOLUTIONS,956379598,2021,09-30-2021,92109.0,0.0,0.0,38652.58333,0.0,459745.50000,70906.416670,1354.666667,569304.50,109559.000000
